In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [ ]:
# 출력시 텍스트 제한 해제
pd.set_option('display.max_colwidth', None)

# 출력시 텍스트 제한
pd.reset_option('display.max_colwidth')

In [ ]:
mainPaths =[ 
    # contents_1
    # './templates/contents/contents_1/귀어귀촌개념.html', 
    # './templates/contents/contents_1/귀어귀촌준비절차.html', 
    # './templates/contents/contents_1/어업면허허가.html', 
    # './templates/contents/contents_1/어업정의.html',

    # contents_2
    # './templates/contents/contents_2/귀어정책지원교육.html', 
    # './templates/contents/contents_2/양식장및어선임대사업.html', 
    # './templates/contents/contents_2/어촌이사혜택.html', 
    # './templates/contents/contents_2/청년어촌정착지원금.html', 

    # # contents_3
    # './templates/contents/contents_3/유해해양생물_종류.html', 
    # './templates/contents/contents_3/유해해양생물_대응법.html', 
    # './templates/contents/contents_3/이상생태현상_굴폐사.html', 
    # './templates/contents/contents_3/이상생태현상_산호초백화.html', 
    # './templates/contents/contents_3/이상생태현상_대응법.html', 

    # # contents_4
    # './templates/contents/contents_4/IoTAI활용스마트양식.html', 
    # './templates/contents/contents_4/친환경지속가능어업.html', 

    # # contents_5
    # './templates/contents/contents_5/어업종류.html', 
    # './templates/contents/contents_5/지역별주력어종.html', 
    # './templates/contents/contents_5/낚시및해루질_금어기.html',
    # './templates/contents/contents_5/낚시및해루질_낚시금지구역.html',
    # './templates/contents/contents_5/낚시및해루질_종류별낚시설명및초보자팁.html', 
    # './templates/contents/contents_5/낚시및해루질_지역별낚시명소.html', 
    # './templates/contents/contents_5/낚시및해루질_지역별낚시용품점.html'
    ]

file_path = mainPaths
contentList = []

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # 총 길이
    chunk_overlap=50,   # 겹치는 길이
    separators=["\n\n", "다.", "요.", ".", "\n", " ", ""]
)



for i in file_path:
    with open(i, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    folder = os.path.basename(os.path.dirname(i)) 
    page = os.path.splitext(os.path.basename(i))[0]  
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = soup.select('[name="section"]')
    titles = None
    contents = None
    title = None
    content = None
    for j in sections:
        titles = j.select('[name="content_title"]')
        contents = j.select('[name="content_text"]')
        # 둘 다 1개 이상일 때
        if len(titles) > 1 and len(contents) > 1:
            count = min(len(titles), len(contents))  # 길이 맞추기
            for k in range(count):
                title = titles[k].get_text(separator=" ",strip=True)
                content = contents[k].get_text(separator=" ", strip=True)
                chunks = splitter.split_text(content)
                for k, chunk in enumerate(chunks):
                    contentList.append([title, chunk, folder, page])

        # 하나만 있을 경우 (정확히 하나일 때만 출력)
        elif len(titles) == 1 and len(contents) == 1:
            title = titles[0].get_text(separator=" ", strip=True)
            content = contents[0].get_text(separator=" ", strip=True)
            chunks = splitter.split_text(content)
            for k, chunk in enumerate(chunks):
                contentList.append([title, chunk, folder, page])
        else:
            print(f"{f}의 제목 또는 내용이 없습니다.")
            # print(j)

df = pd.DataFrame(contentList,columns=["title", "content", "folder", "page"])
df
# len(df['title'].unique())


,title,content,folder,page
0,지역별 주력 어종,"1. 서울특별시\n 어업 유형: 내수면 어업 중심\n 주요 어종: 붕어, 잉어, 메기\n 양식 대상 어종: 메기, 쏘가리\n 2. 부산광역시\n 어업 유형: 연근해 어업 및 양식\n 주요 어종: 고등어, 멸치, 오징어\n 양식 대상 어종: 넙치, 조피볼락\n 3. 대구광역시\n 어업 유형: 내수면 어업 중심\n 주요 어종: 붕어, 잉어, 메기\n 양식 대상 어종: 메기, 쏘가리\n 4. 인천광역시\n 어업 유형: 연근해 어업 및 양식\n 주요 어종: 꽃게, 조기, 전어\n 양식 대상 어종: 넙치, 조피볼락\n 5. 광주광역시\n 어업 유형: 내수면 어업 중심\n 주요 어종: 붕어, 잉어, 메기\n 양식 대상 어종: 메기\n 6",contents_5,지역별주력어종
1,지역별 주력 어종,". 대전광역시\n 어업 유형: 내수면 어업 중심\n 주요 어종: 붕어, 잉어, 메기\n 양식 대상 어종: 메기\n 7. 울산광역시\n 어업 유형: 연근해 어업 및 양식\n 주요 어종: 고등어, 오징어, 전갱이\n 양식 대상 어종: 넙치, 조피볼락\n 8. 세종특별자치시\n 어업 유형: 내수면 어업 중심\n 주요 어종: 붕어, 잉어, 메기\n 양식 대상 어종: 메기\n 9. 경기도\n 어업 유형: 내수면 및 연안 어업\n 주요 어종: 붕어, 잉어, 메기, 꽃게\n 양식 대상 어종: 메기, 쏘가리\n 10. 강원도\n 어업 유형: 연근해 어업 및 양식\n 주요 어종: 명태, 오징어, 대구\n 양식 대상 어종: 대구\n 11",contents_5,지역별주력어종
2,지역별 주력 어종,". 충청북도\n 어업 유형: 내수면 어업 중심\n 주요 어종: 붕어, 잉어, 메기\n 양식 대상 어종: 메기\n 12. 충청남도\n 어업 유형: 연근해 어업 및 양식\n 주요 어종: 꽃게, 조기, 전어\n 양식 대상 어종: 넙치, 조피볼락\n KMI\n 13. 전라북도\n 어업 유형: 연근해 어업 및 양식\n 주요 어종: 전어, 농어, 조기\n 양식 대상 어종: 넙치, 조피볼락\n 14. 전라남도\n 어업 유형: 연근해 어업 및 양식\n 주요 어종: 감성돔, 벵에돔, 전갱이\n 양식 대상 어종: 넙치, 조피볼락, 전복, 김\n 15. 경상북도\n 어업 유형: 연근해 어업 및 양식\n 주요 어종: 대구, 오징어, 명태\n 양식 대상 어종: 대구, 전복\n 16",contents_5,지역별주력어종
3,지역별 주력 어종,". 경상남도\n 어업 유형: 연근해 어업 및 양식\n 주요 어종: 고등어, 멸치, 참다랑어\n 양식 대상 어종: 조피볼락, 벤자리, 전복, 김\n 17. 제주특별자치도\n 어업 유형: 연근해 어업 및 양식\n 주요 어종: 벵에돔, 참돔, 갈치\n 양식 대상 어종: 넙치, 조피볼락, 전복",contents_5,지역별주력어종


In [19]:
# 2. 문서 객체 생성
documents = [
    Document(page_content=row['content'], metadata={"title": row['title'], "folder": row['folder'], "page": row['page']})
    for _, row in df.iterrows()
]

embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)

vectorstore.save_local("chatbot/vector_store/contents")